# 🔍 Notebook Overview
This notebook reads and parses frames from the Waymo Open Dataset using TensorFlow and Protocol Buffers. It then extracts and displays image data and LIDAR information from each frame for visualization or further processing.

## 📦  Imports and Protobuf Definitions



This cell imports:
- TensorFlow: to load TFRecord data.
- Waymo's `dataset_pb2` module: provides access to Protocol Buffers for parsing Waymo frames.
- PIL and NumPy: for handling and converting image data.

In [ ]:
import numpy as np
import tensorflow as tf
import click
import imageio as imageio_v1

from concurrent.futures import ProcessPoolExecutor, as_completed
from pathlib import Path
from typing import List, Dict, Any, Union
from termcolor import cprint
from tqdm import tqdm
from scipy.spatial.transform import Rotation as R
from waymo_open_dataset import dataset_pb2, label_pb2
from waymo_open_dataset.utils import frame_utils
from google.protobuf import json_format

### Load Waymo TFRecord Dataset

Here we load a Waymo `.tfrecord` file using `TFRecordDataset`. The `compression_type=''` indicates the file is uncompressed.



In [ ]:
WaymoProto2SemanticLabel = {
    label_pb2.Label.Type.TYPE_UNKNOWN: "Unknown",
    label_pb2.Label.Type.TYPE_VEHICLE: "Car",
    label_pb2.Label.Type.TYPE_PEDESTRIAN: "Pedestrian",
    label_pb2.Label.Type.TYPE_SIGN: "Sign",
    label_pb2.Label.Type.TYPE_CYCLIST: "Cyclist",
}

CameraNames = ['front', 'front_left', 'front_right', 'side_left', 'side_right']

SourceFps = 10 # waymo's recording fps
TargetFps = 30 # cosmos's expected fps
IndexScaleRatio = int(TargetFps / SourceFps)

if int(tf.__version__.split(".")[0]) < 2:
    tf.enable_eager_execution()

def get_camera_name(name_int) -> str:
    return dataset_pb2.CameraName.Name.Name(name_int)


def get_lidar_name(name_int) -> str:
    return dataset_pb2.LaserName.Name.Name(name_int)

In [ ]:
waymo_tfrecord_root = "/home/dan/development/data_sanity/data-sample"
all_filenames = list(Path(waymo_tfrecord_root).glob("*.tfrecord"))
print(f"Found {len(all_filenames)} tfrecords")

### 🎞️ Convert Waymo Images to Video Frames

This function reads all frames from a Waymo TFRecord dataset and extracts camera images. It organizes the images by camera and saves them as individual JPEG files. These can later be stitched into a video or used independently for analysis or visualization.

---

#### **Function:** `convert_waymo_image(...)`

- **Parameters:**
  - `output_root (Path)`: Directory where the output images will be saved.
  - `clip_id (str)`: Identifier to prefix output filenames.
  - `dataset (tf.data.TFRecordDataset)`: Parsed Waymo TFRecord dataset.
  - `single_camera (bool)`: If `True`, only the 'front' camera is processed.

---

#### **Workflow:**
1. ✅ **Initialize storage** for each camera’s image sequence using `CameraNames`.
2. 🔄 **Iterate through each frame** in the dataset:
   - Parse the serialized frame data into a `Frame` protobuf.
   - Extract each image, decode it from JPEG, convert to NumPy, and append to the corresponding camera's list.
3. 💾 **Save extracted images**:
   - If `single_camera=True`, skip all but the 'front' camera.
   - Write each image to disk in the format:  
     `pinhole_<camera_name>/<clip_id>_<frame_index>.jpg`.
   - Create output directories as needed.

---

This function assumes Waymo cameras operate at **10 Hz**, meaning each second of footage yields 10 frames per camera.


In [ ]:
def convert_waymo_image(output_root: Path, clip_id: str, dataset: tf.data.TFRecordDataset, single_camera: bool = False  ):
    """
    read all frames and convert the images to video format.
    """
    cprint('reading image and converting to video, this may take a while...', 'yellow')

    camera_name_to_image_sequence = {}
    for camera_name in CameraNames:
        camera_name_to_image_sequence[camera_name] = []

    for frame_idx, data in enumerate(dataset):
        frame = dataset_pb2.Frame()
        frame.ParseFromString(bytes(data.numpy()))

        for image_data in frame.images:
            camera_name = get_camera_name(image_data.name).lower()
            image_data_bytes = image_data.image
            image_data_tf_tensor = tf.image.decode_jpeg(image_data_bytes)
            image_data_numpy = image_data_tf_tensor.numpy()
            camera_name_to_image_sequence[camera_name].append(image_data_numpy)
    
    for camera_name, image_sequence in camera_name_to_image_sequence.items():
        # waymo is recorded at 10 Hz
        if single_camera and camera_name != 'front':
           continue

        for i,image in enumerate(image_sequence):
            output_image_path = (output_root / f"pinhole_{camera_name}" / f'{clip_id}_{i}.jpg')
            output_image_path.parent.mkdir(parents=True, exist_ok=True)
            imageio_v1.imwrite(output_image_path, image)


### 🌐 Convert Waymo LIDAR to Cosmos Format

This function processes each frame in a Waymo TFRecord dataset to extract LIDAR point clouds and saves them in `.pcd` format using Open3D.

---

#### **Function:** `convert_waymo_lidar(...)`

- **Parameters:**
  - `output_root (Path)`: Directory where the LIDAR point clouds will be saved.
  - `clip_id (str)`: Identifier used to prefix output files.
  - `dataset (tf.data.TFRecordDataset)`: Waymo TFRecord dataset.

---



In [ ]:
import open3d as o3d

def convert_waymo_lidar(output_root: Path, clip_id: str, dataset: tf.data.TFRecordDataset):
    """
    read all frames and convert the lidar

    """
    sample = {'__key__': clip_id}
    cprint('reading lidar and converting to cosmos format, this may take a few minutes...', 'yellow')

    for frame_idx, data in enumerate(dataset):
        frame = dataset_pb2.Frame()
        frame.ParseFromString(bytes(data.numpy()))

        vehicle_to_world = np.array(frame.pose.transform).reshape((4, 4))
        
        range_images, camera_projections,  _, range_image_top_pose = \
            frame_utils.parse_range_image_and_camera_projection(frame)

        points, _ = frame_utils.convert_range_image_to_point_cloud(
            frame, range_images, camera_projections, range_image_top_pose
        )
        points_ri2, _ = frame_utils.convert_range_image_to_point_cloud(
            frame, range_images, camera_projections, range_image_top_pose, ri_index=1
        )

        # 3d points in vehicle frame. 
        points = [np.concatenate([p1, p2]) for p1, p2 in zip(points, points_ri2)] 

        lidar_ids = [calib.name for calib in frame.context.laser_calibrations]
        lidar_ids.sort()

        for lidar_id, lidar_points in zip(lidar_ids, points):
            lidar_name = get_lidar_name(lidar_id)

            if lidar_name == 'TOP':
                # Convert lidar points to Open3D PointCloud format
                pcd = o3d.geometry.PointCloud()
                pcd.points = o3d.utility.Vector3dVector(lidar_points)

                # Define the output path for the PCD file
                pcd_output_path = output_root / 'lidar_pcd' / f"{clip_id}_{frame_idx * IndexScaleRatio}.pcd"
                pcd_output_path.parent.mkdir(parents=True, exist_ok=True)

                # Save the PointCloud to a PCD file
                o3d.io.write_point_cloud(str(pcd_output_path), pcd)


### 🛠️ Waymo Dataset Initialization and Configuration

This block sets up paths, parameters, and the data loading pipeline needed to process a Waymo TFRecord file for conversion into image and LIDAR formats.

---

#### **📷 Camera Setup**
Defines the list of camera names present in the Waymo dataset:
```python
CameraNames = ['front', 'front_left', 'front_right', 'side_left', 'side_right']
```

### Unpacking the TFRecord
This block also will unpack a `tfrecord` into `jpg`s and `pcd`s

In [ ]:
CameraNames = ['front', 'front_left', 'front_right', 'side_left', 'side_right']

SourceFps = 10 # waymo's recording fps
TargetFps = 30 # cosmos's expected fps
IndexScaleRatio = int(TargetFps / SourceFps)

waymo_tfrecord_root = "/home/dan/development/data_sanity/data-sample/"
all_filenames = list(Path(waymo_tfrecord_root).glob("*.tfrecord"))
print(f"Found {len(all_filenames)} tfrecords")

single_camera = False

waymo_tfrecord_filename = all_filenames[0].name
output_data_path = "/home/dan/development/data_sanity/data-sample/fiftyone" 
waymo_tfrecord_path = Path(waymo_tfrecord_root + waymo_tfrecord_filename)
clip_id = waymo_tfrecord_path.stem.lstrip('segment-').rstrip('_with_camera_labels')
output_data_path = Path(output_data_path)

if not waymo_tfrecord_path.exists():
    raise FileNotFoundError(f"Waymo tfrecord file not found: {waymo_tfrecord_path}")


tf_dataset = tf.data.TFRecordDataset(waymo_tfrecord_path, compression_type="")


convert_waymo_image(output_data_path, clip_id, tf_dataset, single_camera)
convert_waymo_lidar(output_data_path, clip_id, tf_dataset)


## Creating FiftyOne Dataset

This code initializes a FiftyOne dataset named `"Waymo-Open-Dataset-Test"` and populates it with grouped samples composed of camera images and LIDAR point clouds extracted from a Waymo TFRecord export. It begins by loading all front-facing camera images from the output directory, sorting them by frame index. For each image, it constructs a sample group that may include additional camera views (if `single_camera` is `False`) and a corresponding `.pcd` LIDAR file if available.

Each image is added as an individual `fo.Sample`, assigned a sensor name and grouped by frame. If LIDAR data is present, it is wrapped in a `fo.Scene` object, written to disk as a `.fo3d` file, and added as a 3D sample to the group. All samples are appended to a list and finally registered with the FiftyOne dataset. The result is a synchronized, multi-sensor dataset where each frame includes images from multiple cameras and an optional LIDAR scene, viewable and queryable in FiftyOne.

See [FiftyOne Grouped Datasets](https://docs.voxel51.com/user_guide/groups.html) for more info!


In [ ]:
import fiftyone as fo
fo_dataset = fo.Dataset(name="Waymo-Open-Dataset-Test", persistent=True, overwrite=True)

image_dir = output_data_path / "pinhole_front"
image_files = list(image_dir.glob("*.jpg"))
print(f"Found {len(image_files)} images in {image_dir}")
image_files.sort(key=lambda x: int(x.stem.split('_')[-1]))

samples = []
for image_file in image_files:
    
    group = fo.Group()
    frame = image_file.stem.split('_')[-1]
    clip_id = "_".join(image_file.stem.split('_')[:5])
    sensors = {}
    sensors['pinhole_front'] = {
        'file': image_file,
        'frame': int(frame),
        'clip_id': clip_id,
        'sensor_name': 'front',
        'modality': 'image'
    }
    if not single_camera:
        for camera_name in CameraNames:
            if camera_name != 'front':
                other_camera_dir = output_data_path / f"pinhole_{camera_name}/"
                other_camera_file = str(other_camera_dir) + f"/{clip_id}_{frame}.jpg"
                
                if not Path(other_camera_file).exists():
                    print(f"File not found: {other_camera_file}")
                    continue
                else:
                    sensors[f'pinhole_{camera_name}'] = {
                        'file': other_camera_file,
                        'frame': int(frame),
                        'clip_id': clip_id,
                        'sensor_name': camera_name,
                        'modality': 'image'
                    }
    lidar_dir = output_data_path / f"lidar_pcd/"
    lidar_file = str(lidar_dir) + f"/{clip_id}_{frame}.pcd"
    if Path(lidar_file).exists():
        sensors['lidar'] = {
            'file': lidar_file,
            'frame': int(frame),
            'clip_id': clip_id,
            'sensor_name': 'lidar_top',
            'modality': 'pointcloud'
        }

    for sensor_name, sensor_data in sensors.items():
        fo3d_scene = fo.Scene(camera=fo.PerspectiveCamera(up="Z"))
        fo3d_output_dir = output_data_path / f"fo3d/"
        fo3d_filepath = str(fo3d_output_dir) + f"/{clip_id}_{frame}.fo3d"
        Path(fo3d_output_dir).mkdir(parents=True, exist_ok=True)

        if sensor_data["modality"] == "image":
            sample = fo.Sample(
                filepath=sensor_data['file'],
                frame=sensor_data['frame'],
                clip_id=sensor_data['clip_id'],
                sensor_name=sensor_data['sensor_name'],
                group=group.element(sensor_name)
            )
            samples.append(sample)
        else:
            fo3d_scene.add(
                    fo.PointCloud(
                        name=sensor_name,
                        pcd_path=sensor_data['file'],
                        flag_for_projection=sensor_name == "lidar_top",
                    )
                )
    fo3d_scene.write(fo3d_filepath)
    fo3d_sample = fo.Sample(filepath=fo3d_filepath, group=group.element("3D"))
    fo3d_sample["clip_id"] = sensor_data['clip_id']
    fo3d_sample["frame"] = sensor_data['frame']
    fo3d_sample["sensor_name"] = "lidar_top"
    samples.append(fo3d_sample)

fo_dataset.add_samples(samples)

In [ ]:
session = fo.launch_app(fo_dataset, port=2830)

# Labels and Metadata

Now that we have gotten through the core media in jpgs and pcds, its time to add the rest! Note that some of this is still a work in progress for one reason or another. Here is a comprehensive list of what exists here:

- `convert_waymo_hdmap` (works, but bugged in latest FO release )
- `convert_waymo_intrinsics` (works!)
- `convert_waymo_pose` (works!)
- `convert_waymo_timestamp` (works!)
- `convert_waymo_3dbbox` (bugged)
- `convert_waymo_2dbbox` (works!)


In [ ]:
def convert_waymo_hdmap(sample, clip_id: str, dataset: tf.data.TFRecordDataset, ):
    """
    read the first frame and convert the hdmap to wds format.
    """
    def hump_to_underline(hump_str):
        import re
        return re.sub(r'([a-z])([A-Z])', r'\1_\2', hump_str).lower()

    hdmap_names_polyline = ["lane", "road_line", "road_edge"]
    hdmap_names_polygon = ["crosswalk", "speed_bump", "driveway"]
    
    hdmap_name_to_data = {}
    for hdmap_name in hdmap_names_polyline + hdmap_names_polygon:
        hdmap_name_to_data[hump_to_underline(hdmap_name)] = []

    for frame_idx, data in enumerate(dataset):
        if frame_idx == sample.frame:
            frame = dataset_pb2.Frame()
            frame.ParseFromString(bytes(data.numpy()))

            map_features_list = json_format.MessageToDict(frame)['mapFeatures']

            for hdmap_content in map_features_list:
                hdmap_name = list(hdmap_content.keys())
                hdmap_name.remove("id")
                hdmap_name = hdmap_name[0]
                hdmap_name_lower = hump_to_underline(hdmap_name)

                hdmap_data = hdmap_content[hdmap_name]
                if hdmap_name_lower in hdmap_names_polyline:
                    hdmap_data = hdmap_data['polyline']
                    polyline = [[point['x'], point['y'], point['z']] for point in hdmap_data]
                    hdmap_name_to_data[hdmap_name_lower].append(polyline)
                elif hdmap_name_lower in hdmap_names_polygon:
                    hdmap_data = hdmap_data['polygon']
                    polygon = [[point['x'], point['y'], point['z']] for point in hdmap_data]
                    hdmap_name_to_data[hdmap_name_lower].append(polygon)
                else:
                    print(f"Unkown hdmap item name: {hdmap_name}，skip this item")


    # convert to cosmos's name convention for easier processing
    hdmap_name_to_cosmos = {
        'lane': 'lanes',
        'road_line': 'lanelines',
        'road_edge': 'road_boundaries',
        'crosswalk': 'crosswalks',
        'speed_bump': None,
        'driveway': None
    }

    polylines = []
    for hdmap_name, hdmap_data in hdmap_name_to_data.items():
        hdmap_name_in_cosmos = hdmap_name_to_cosmos[hdmap_name]
        if hdmap_name_in_cosmos is None:
            continue

        if hdmap_name in hdmap_names_polyline:
            vertex_indicator = 'polyline3d'
        else:
            vertex_indicator = 'surface'


        for each_polyline_or_polygon in hdmap_data:

            if each_polyline_or_polygon is None or len(each_polyline_or_polygon) == 0:
                print(f"Skipping empty {hdmap_name} data")
                continue

            polyline = fo.Polyline(label=hdmap_name, points3d=each_polyline_or_polygon)
            return each_polyline_or_polygon
            polylines.append(polyline)
    
    sample["polylines"] = fo.Polylines(polylines=polylines)
    sample.save()


In [ ]:
def convert_waymo_intrinsics(sample, clip_id: str, dataset: tf.data.TFRecordDataset,):
    """
    read the first frame and convert the intrinsics to wds format

    Minimal required format:
        sample['pinhole_intrinsic.{camera_name}.npy'] = np.ndarray with shape (4, 4)
    """

    for frame_idx, data in enumerate(dataset):
         if frame_idx == sample.frame:
            
            frame = dataset_pb2.Frame()
            frame.ParseFromString(bytes(data.numpy()))

            for camera_calib in frame.context.camera_calibrations:
                camera_name = get_camera_name(camera_calib.name).lower()
                if camera_name == sample.sensor_name:
                    intrinsic = camera_calib.intrinsic
                    fx, fy, cx, cy = intrinsic[:4]
                    w, h = camera_calib.width, camera_calib.height

                    sample["camera_intrinsics"] = \
                        np.array([fx, fy, cx, cy, w, h])
                    sample.save()


In [ ]:
from fiftyone import ViewField as F
def interpolate_pose(prev_pose, next_pose, t):
    """
    new pose = (1 - t) * prev_pose + t * next_pose.
    - linear interpolation for translation
    - slerp interpolation for rotation

    Args:
        prev_pose: np.ndarray, shape (4, 4), dtype=np.float32, previous pose
        next_pose: np.ndarray, shape (4, 4), dtype=np.float32, next pose
        t: float, interpolation factor

    Returns:
        np.ndarray, shape (4, 4), dtype=np.float32, interpolated pose

    Note:
        if input is list, also return list.
    """
    input_is_list = isinstance(prev_pose, list)
    prev_pose = np.array(prev_pose)
    next_pose = np.array(next_pose)

    prev_translation = prev_pose[:3, 3]
    next_translation = next_pose[:3, 3]
    translation = (1 - t) * prev_translation + t * next_translation

    prev_rotation = R.from_matrix(prev_pose[:3, :3])
    next_rotation = R.from_matrix(next_pose[:3, :3])
    
    times = [0, 1]
    rotations = R.from_quat([prev_rotation.as_quat(), next_rotation.as_quat()])
    rotation = Slerp(times, rotations)(t)

    new_pose = np.eye(4)
    new_pose[:3, :3] = rotation.as_matrix()
    new_pose[:3, 3] = translation

    if input_is_list:
        return new_pose.tolist()
    else:
        return new_pose
    
def convert_waymo_pose(fo_dataset, clip_id: str, dataset: tf.data.TFRecordDataset):
    """
    read all frames and convert the pose to wds format. interpolate the pose to the target fps

    Minimal required format:
        sample_camera_to_world['{frame_idx:06d}.pose.{camera_name}.npy'] = np.ndarray with shape (4, 4). opencv convention
        sample_vehicle_to_world['{frame_idx:06d}.vehicle_pose.npy'] = np.ndarray with shape (4, 4). flu convention
    """

    camera_name_to_camera_to_vehicle = {}
    

    # get camera_to_vehicle from the first frame
    for frame_idx, data in enumerate(dataset):
        frame = dataset_pb2.Frame()
        frame.ParseFromString(bytes(data.numpy()))

        for camera_calib in frame.context.camera_calibrations:
            camera_name = get_camera_name(camera_calib.name).lower()
            camera_to_vehicle = np.array(camera_calib.extrinsic.transform).reshape((4, 4)) # FLU convention
            camera_name_to_camera_to_vehicle[camera_name] = camera_to_vehicle

        # only process the first frame
        break

    for frame_idx, data in enumerate(dataset):
        frame = dataset_pb2.Frame()
        frame.ParseFromString(bytes(data.numpy()))

        group = fo_dataset.get_group(fo_dataset.match(F("frame")==frame_idx).first().group.id)
        sample = group["3D"]
        sample = fo_dataset[sample.filepath]
        sample["vehicle_to_world"] = np.array(frame.pose.transform).reshape((4, 4))
        sample.save()

        for image_data in frame.images:
            camera_name = get_camera_name(image_data.name).lower()
            sample = group["pinhole_" + camera_name]
            sample["vehicle_to_world"] = np.array(image_data.pose.transform).reshape((4, 4))
            sample["camera_to_vehicle"] = camera_name_to_camera_to_vehicle[camera_name]
            sample["camera_to_world"] = sample["vehicle_to_world"] @ camera_to_vehicle # FLU convention
            sample["camera_to_world_opencv"] = np.concatenate(
                [-sample["camera_to_world"][:, 1:2], -sample["camera_to_world"][:, 2:3], sample["camera_to_world"][:, 0:1], sample["camera_to_world"][:, 3:4]],
                axis=1
            )
            sample.save()


    # interpolate the pose to the target fps
    # source index: 0,    1,    2,    3, ..., 10
    # target index: 0,1,2,3,4,5,6,7,8,9, ..., 30,31,32
    max_target_frame_idx = frame_idx * IndexScaleRatio

    # interpolate the vehicle pose to the target fps
    for target_frame_idx in range(max_target_frame_idx):
        group = fo_dataset.get_group(fo_dataset.match(F("frame")==frame_idx).first().group.id)
        if group["pinhole_" + camera_name]["vehicle_to_world"] is  None:
            nearest_prev_frame_idx = target_frame_idx // IndexScaleRatio * IndexScaleRatio
            nearest_prev_frame_pose = fo_dataset.get_group(fo_dataset.match(F("frame")==nearest_prev_frame_idx).first().group.id)["pinhole_" + camera_name]["vehicle_to_world"]
            nearest_next_frame_idx = (target_frame_idx // IndexScaleRatio + 1) * IndexScaleRatio
            nearest_next_frame_pose = fo_dataset.get_group(fo_dataset.match(F("frame")==nearest_next_frame_idx).first().group.id)["pinhole_" + camera_name]["vehicle_to_world"]
            
            int_pose = \
                interpolate_pose(nearest_prev_frame_pose, nearest_next_frame_pose, (target_frame_idx - nearest_prev_frame_idx) / IndexScaleRatio)
            for _, sample in group.items():
                sample["vehicle_to_world"] = int_pose
                sample.save()



    # interpolate the camera pose to the target fps
    for camera_name in CameraNames:
        for target_frame_idx in range(max_target_frame_idx):
            group = fo_dataset.get_group(fo_dataset.match(F("frame")==frame_idx).first().group.id)
            if group["pinhole_" + camera_name]["camera_to_world"] is  None:
                nearest_prev_frame_idx = target_frame_idx // IndexScaleRatio * IndexScaleRatio
                nearest_prev_frame_pose = fo_dataset.get_group(fo_dataset.match(F("frame")==nearest_prev_frame_idx).first().group.id)["pinhole_" + camera_name]["camera_to_world"]
                nearest_next_frame_idx = (target_frame_idx // IndexScaleRatio + 1) * IndexScaleRatio
                nearest_next_frame_pose = fo_dataset.get_group(fo_dataset.match(F("frame")==nearest_next_frame_idx).first().group.id)["pinhole_" + camera_name]["camera_to_world"]
                
                int_pose = \
                    interpolate_pose(nearest_prev_frame_pose, nearest_next_frame_pose, (target_frame_idx - nearest_prev_frame_idx) / IndexScaleRatio)
                sample = group["pinhole_" + camera_name]
                sample["camera_to_world"] = int_pose
                sample.save()

        

In [ ]:
def convert_waymo_timestamp(fo_dataset, clip_id: str, dataset: tf.data.TFRecordDataset):
    """
    read all frames and convert the timestamp to wds format.
    """
    
    for frame_idx, data in enumerate(dataset):
        frame = dataset_pb2.Frame()
        frame.ParseFromString(bytes(data.numpy()))
        timestamp_micros = frame.timestamp_micros
        group = fo_dataset.get_group(fo_dataset.match(F("frame")==frame_idx).first().group.id)
        for _, sample in group.items():
                sample["timestamp"] = timestamp_micros
                sample.save()


In [ ]:
#TODO
def convert_waymo_3dbbox(fo_dataset, clip_id: str, dataset: tf.data.TFRecordDataset):
    """
    read all frames and convert the 3dbbox 

    """

    min_moving_speed = 0.2

    valid_bbox_types = [
        label_pb2.Label.Type.TYPE_VEHICLE,
        label_pb2.Label.Type.TYPE_PEDESTRIAN,
        label_pb2.Label.Type.TYPE_CYCLIST
    ]

    for frame_idx, data in enumerate(dataset):
        frame = dataset_pb2.Frame()
        frame.ParseFromString(bytes(data.numpy()))
        group = fo_dataset.get_group(fo_dataset.match(F("frame")==frame_idx).first().group.id)

        vehicle_to_world = np.array(frame.pose.transform).reshape((4, 4))

        detections = []
        for label in frame.laser_labels:
            if label.type not in valid_bbox_types:
                continue

            if not label.camera_synced_box.ByteSize():
                continue

            object_id = label.id
            object_type = WaymoProto2SemanticLabel[label.type]

            center_in_vehicle = np.array([label.camera_synced_box.center_x, label.camera_synced_box.center_y, label.camera_synced_box.center_z, 1]).reshape((4, 1))
            center_in_world = vehicle_to_world @ center_in_vehicle
            heading = label.camera_synced_box.heading
            rotation_in_vehicle = R.from_euler("xyz", [0, 0, heading], degrees=False).as_matrix()
            rotation_in_world = vehicle_to_world[:3, :3] @ rotation_in_vehicle

            object_to_world = np.eye(4)
            object_to_world[:3, :3] = rotation_in_world
            object_to_world[:3, 3] = center_in_world.flatten()[:3]

            object_lwh = np.array([label.camera_synced_box.length, label.camera_synced_box.width, label.camera_synced_box.height])
            
            speed = np.sqrt(label.metadata.speed_x**2 + label.metadata.speed_y**2 + label.metadata.speed_z**2)
            object_is_moving = bool(speed > min_moving_speed)
            center_in_world = center_in_world.flatten()[:3]
            print(center_in_vehicle.flatten()[:3])
            detection = fo.Detection(
                label=object_type,
                location=center_in_world,
                dimensions=object_lwh,
                rotation=R.from_matrix(rotation_in_vehicle).as_euler('xyz', degrees=False),
                )
            detections.append(detection)
        
        sample = group["3D"]
        detections.append(fo.Detection(
            label="vehicle",
            location=[0,0,0],
            dimensions=[3,3,3],
            rotation=[0, 0, 0],
        ))
        sample["detections"] = fo.Detections(detections=detections)
        sample.save()


In [ ]:
def convert_waymo_2dbbox(fo_dataset, tf_dataset: tf.data.TFRecordDataset):
    for frame_idx, data in enumerate(tf_dataset):
        frame = dataset_pb2.Frame()
        frame.ParseFromString(bytes(data.numpy()))
        group = fo_dataset.get_group(fo_dataset.match(F("frame")==frame_idx).first().group.id)
        for camera_labels in frame.camera_labels:
        # Ignore camera labels that do not correspond to this camera.
            sample = group["pinhole_" + get_camera_name(camera_labels.name).lower()]
            detections = []
            for label in camera_labels.labels:
                x = (label.box.center_x - 0.5 * label.box.length)/sample.metadata.width
                y = (label.box.center_y - 0.5 * label.box.width)/ sample.metadata.height
                width = label.box.length / sample.metadata.width
                height = label.box.width / sample.metadata.height
                label = WaymoProto2SemanticLabel[label.type]
                det = fo.Detection(
                    label=label,
                    bounding_box=[x, y, width, height],
                )
                detections.append(det)
            sample["2d_detections"] = fo.Detections(detections=detections)
            sample.save()
                
                

Need to compute metadata first!

In [ ]:
fo_dataset.compute_metadata()

In [ ]:
convert_waymo_2dbbox(fo_dataset, tf_dataset)

In [ ]:
image_view = fo_dataset.select_group_slices(media_type="image")
for sample in image_view:
    convert_waymo_intrinsics(sample, clip_id, tf_dataset)

In [ ]:
convert_waymo_pose(fo_dataset, clip_id, tf_dataset)

In [ ]:
convert_waymo_timestamp(fo_dataset, clip_id, tf_dataset)

In [ ]:
convert_waymo_3dbbox(fo_dataset, clip_id, tf_dataset)

In [ ]:
fo_dataset.export(
    export_dir="fo_waymo_dataset",
    dataset_type=fo.types.FiftyOneDataset,
)